In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

# custom module
import sys

sys.path.append("../../../")  # include parent directory
from src.ookla import *

/home/abbymoreno/miniconda3/envs/lacuna-fund2/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
ADMIN_FPATH = Path("../../../data/01-admin-bounds/")
PROCESSED_FPATH = Path("../../../data/03-processed/ookla/")
OOKLA_DIR = Path("../../../data/02-raw/ookla/")

### Load AOI

In [4]:
# Load the administrative boundaries for the 12 cities
aoi = gpd.read_file(ADMIN_FPATH / "renamed_target_admin_bounds.gpkg")
aoi.head(2)

,region_name,region_code,province_name,province_code,city_name,city_code,barangay_name,barangay_psgc_code,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


# Align Ookla data to AOI

`OOKLA_DIR` contains the downloaded raw ookla files from the previous notebook.

In [6]:
ookla_data_manager = OoklaDataManager(cache_dir=OOKLA_DIR)
ookla_features = add_ookla_features(aoi, "fixed", "2022", ookla_data_manager)
ookla_features.head(2)

2023-08-09 11:45:16.450 | DEBUG    | src.ookla:load_type_year_data:76 - Contents of data cache: []
2023-08-09 11:45:16.453 | INFO     | src.ookla:load_type_year_data:91 - Cached data available at ../../../data/02-raw/ookla/ookla/processed/afc08259ba9585cdf5ee4494d1e49fbd.csv? True
2023-08-09 11:45:16.454 | DEBUG    | src.ookla:load_type_year_data:96 - Processed Ookla data for aoi, fixed 2022 (key: afc08259ba9585cdf5ee4494d1e49fbd) found in filesystem. Loading in cache.


,region_name,region_code,province_name,province_code,city_name,city_code,barangay_name,barangay_psgc_code,geometry,fixed_2022_mean_avg_d_kbps_mean,fixed_2022_mean_avg_u_kbps_mean,fixed_2022_mean_avg_lat_ms_mean,fixed_2022_mean_num_tests_mean,fixed_2022_mean_num_devices_mean
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",2586.706255,1843.592792,2.321099,1.311166,0.230061
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",6443.622866,6426.562701,1.205272,21.050825,4.938126


In [7]:
ookla_features_mobile = add_ookla_features(aoi, "mobile", "2022", ookla_data_manager)
ookla_features_mobile.head(2)

2023-08-09 11:45:31.997 | DEBUG    | src.ookla:load_type_year_data:76 - Contents of data cache: ['afc08259ba9585cdf5ee4494d1e49fbd']
2023-08-09 11:45:31.999 | INFO     | src.ookla:load_type_year_data:91 - Cached data available at ../../../data/02-raw/ookla/ookla/processed/588422765f080d117cc64adbde2a3582.csv? True
2023-08-09 11:45:32.000 | DEBUG    | src.ookla:load_type_year_data:96 - Processed Ookla data for aoi, mobile 2022 (key: 588422765f080d117cc64adbde2a3582) found in filesystem. Loading in cache.


,region_name,region_code,province_name,province_code,city_name,city_code,barangay_name,barangay_psgc_code,geometry,mobile_2022_mean_avg_d_kbps_mean,mobile_2022_mean_avg_u_kbps_mean,mobile_2022_mean_avg_lat_ms_mean,mobile_2022_mean_num_tests_mean,mobile_2022_mean_num_devices_mean
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",500.613845,152.336423,3.169237,0.226313,0.166182
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",3246.477497,946.994351,2.699362,3.736913,2.158613


In [15]:
drop_cols = [
    "ADM1_EN",
    "ADM1_PCODE",
    "ADM2_EN",
    "ADM2_PCODE",
    "ADM3_EN",
    "ADM3_PCODE",
    "ADM4_EN",
    "geometry",
]
ookla_features = ookla_features.drop(columns=drop_cols)
ookla_features_mobile = ookla_features_mobile.drop(columns=drop_cols)

In [19]:
ookla_features.to_csv(PROCESSED_FPATH / "ookla_features_fixed_2022.csv")

In [20]:
ookla_features_mobile.to_csv(PROCESSED_FPATH / "ookla_features_mobile_2022.csv")